In [1]:
import logging
import geopandas as gpd
# from sliderule import icesat2
from sliderule import sliderule, icesat2
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import geopandas as gpd
from sliderule import gedi, earthdata, raster
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import rasterio
from pyproj import Transformer

In [2]:
region_name='Rift'

Xmin = 2147598.266601563
Ymin = 649070.7397460939
Xmax = 2161712.6464843755
Ymax = 663642.8833007814


X= np.arange(Xmin,Xmax,100)  # X from Xmin to Xmax with a step of 100
Y = np.arange( Ymin,Ymax,100)  # Y from Ymin to Ymax  with a step of 100

In [3]:
from pyproj import Transformer


# Define the transformer from EPSG:3031 (Polar Stereographic) to EPSG:4326 (Lat/Lon)
transformer = Transformer.from_crs("EPSG:3031", "EPSG:4326", always_xy=True)

# Convert all (X, Y) pairs
lon, lat = transformer.transform(*np.meshgrid(X, Y))

# Get the min and max lat/lon values
lat_min, lat_max = lat.min(), lat.max()
lon_min, lon_max = lon.min(), lon.max()

# Output the range
lat_min, lat_max, lon_min, lon_max

lon_up = lon_max
lat_up = lat_max
lon_lo = lon_min
lat_lo = lat_min


# Configure ICESat-2 API
icesat2.init("slideruleearth.io")
# Configure Region of Interest
region = [ {"lon":lon_up, "lat": lat_up},
           {"lon":lon_lo, "lat": lat_up},
           {"lon":lon_lo, "lat": lat_lo},
           {"lon":lon_up, "lat": lat_lo},
           {"lon":lon_up, "lat": lat_up} ]


# Build ATL06 Request
 #calculates geolocated elevations in the selected region at a 20m along-track resolution.
length=10.0#40.0,30.0,20.0,10.0,5.0

parms = {
    "poly": region,
    "srt": 3,#3 (land ice)
    "cnf": 4,
    "ats": 20.0,
    "cnt": 10,
    "len": length,
    "res": 20.0
}
# Request ATL06 Data
gdf6 = icesat2.atl06p(parms)


In [4]:
# Convert from EPSG:7912 to Polar Stereographic
# First, ensure your gdf6 has geometries in geodetic lat/lon (EPSG:4326)
gdf6 = gdf6.to_crs(epsg=4326)
# reproject to the Antarctica polar projection
gdf6 = gdf6.to_crs(epsg=3031)
print(gdf6.crs)

EPSG:3031


In [5]:
# # dem_file ="/home/jovyan/Glacier_preprocessing/REMA Map Poster Blank_modified2.tif"
# dem_file ="/home/jovyan/Glacier_preprocessing/LC09_L1GT_128109_20241231_20241231_02_T2_B8.TIF"

# # Open the DEM file
# with rasterio.open(dem_file) as src:
#     antarctic_dem = src.read(1)  # Read the first band
#     extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]


# # Plot the DEM with the updated zoom-in
# fig, ax = plt.subplots(figsize=(12, 6))
# im = ax.imshow(antarctic_dem, extent=extent, cmap="gray")

# # Plot `gdf6['h_mean']` as colored scatter points
# scatter = ax.scatter(
#     gdf6.geometry.x, gdf6.geometry.y, 
#     c=gdf6["h_mean"], cmap="coolwarm", s=1.0, alpha=0.8)


# # Add colorbar for `h_mean`
# plt.colorbar(scatter, label="h_mean (m)")

# # Formatting
# plt.title(f'{region_name} of Antarctic DEM with h_mean Overlay for length {length}')
# plt.xlabel("X (EPSG:3031)")
# plt.ylabel("Y (EPSG:3031)")

# plt.show()

In [6]:
# # Open the DEM file
# with rasterio.open(dem_file) as src:
#     antarctic_dem = src.read(1)  # Read the first band
#     extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# # Get bounding box of `gdf6` scatter points
# x_min, x_max = gdf6.geometry.x.min(), gdf6.geometry.x.max()
# y_min, y_max = gdf6.geometry.y.min(), gdf6.geometry.y.max()

# # Expand the bounding box slightly (by 10% buffer)
# x_buffer = (x_max - x_min) * 0.1
# y_buffer = (y_max - y_min) * 0.1

# x_min -= x_buffer
# x_max += x_buffer
# y_min -= y_buffer
# y_max += y_buffer

# x_min -= 50000  # Expand by 9 km
# x_max += 50000
# y_min -= 50000
# y_max += 50000

# # Plot the DEM with the updated zoom-in
# # fig, ax = plt.subplots(figsize=(40, 30), dpi=600)
# fig, ax = plt.subplots(figsize=(40, 30), dpi=400)
# im = ax.imshow(antarctic_dem, extent=extent, cmap="gray")

# # # Plot `gdf6['h_mean']` as colored scatter points
# # scatter = ax.scatter(
# #     gdf6.geometry.x, gdf6.geometry.y, 
# #     c=gdf6["h_mean"], cmap="coolwarm", s=1.0, alpha=0.7
# # )

# scatter = ax.scatter(
#     gdf6.geometry.x, gdf6.geometry.y, 
#     c=gdf6["h_mean"], cmap="coolwarm_r", s=5.0, alpha=0.7
# )
# # Use the scatter plot object for colorbar
# cbar = plt.colorbar(scatter, label="h_mean (m)")

# # Set zoom limits
# # ax.set_xlim([x_min, x_max])
# # ax.set_ylim([y_min, y_max])

# ax.set_xlim([2137598,2180000])
# ax.set_ylim([640000,680000])



# cbar.ax.tick_params(labelsize=30)  # Increase colorbar tick labels
# cbar.set_label("h_mean (m)", fontsize=50)  # Increase colorbar label size

# # Increase axis labels and title size
# ax.set_title(f'{region_name} zoomed-In View of Antarctic DEM with h_mean Overlay for length {length}', fontsize=50)  # Increase title size
# ax.set_xlabel("X", fontsize=50)  # Increase x-axis label size
# ax.set_ylabel("Y", fontsize=50)  # Increase y-axis label size

# # Increase tick label size
# ax.tick_params(axis='both', which='major', labelsize=30)

# plt.show()


In [7]:
# # Open the DEM file
# with rasterio.open(dem_file) as src:
#     antarctic_dem = src.read(1)  # Read the first band
#     extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# # Get bounding box of `gdf6` scatter points
# x_min, x_max = gdf6.geometry.x.min(), gdf6.geometry.x.max()
# y_min, y_max = gdf6.geometry.y.min(), gdf6.geometry.y.max()

# # Expand the bounding box slightly (by 10% buffer)
# x_buffer = (x_max - x_min) * 0.1
# y_buffer = (y_max - y_min) * 0.1

# x_min -= x_buffer
# x_max += x_buffer
# y_min -= y_buffer
# y_max += y_buffer

# x_min -= 50000  # Expand by 9 km
# x_max += 540000
# y_min -= 50000
# y_max += 50000

# # Plot the DEM with the updated zoom-in
# # fig, ax = plt.subplots(figsize=(40, 30), dpi=600)
# fig, ax = plt.subplots(figsize=(40, 30), dpi=400)
# im = ax.imshow(antarctic_dem, extent=extent, cmap="gray")

# # # Plot `gdf6['h_mean']` as colored scatter points
# scatter = ax.scatter(
#    subset.geometry.x, subset.geometry.y, 
#     c=subset["h_mean"], cmap="coolwarm", s=15, alpha=0.7
# )
# # Use the scatter plot object for colorbar
# cbar = plt.colorbar(scatter, label="h_mean (m)")

# # # Set zoom limits
# ax.set_xlim([x_min, 2217614.348658402])
# ax.set_ylim([y_min, y_max])

# # ax.set_xlim([-615000, -604000])
# # ax.set_ylim([42000, 53000])

# cbar.ax.tick_params(labelsize=30)  # Increase colorbar tick labels
# cbar.set_label("h_mean (m)", fontsize=50)  # Increase colorbar label size

# # Increase axis labels and title size
# ax.set_title(f'{region_name} zoomed-In View of Antarctic DEM with h_mean Overlay for length {length}', fontsize=50)  # Increase title size
# ax.set_xlabel("X", fontsize=50)  # Increase x-axis label size
# ax.set_ylabel("Y", fontsize=50)  # Increase y-axis label size

# # Increase tick label size
# ax.tick_params(axis='both', which='major', labelsize=30)

# plt.show()


In [8]:
# # subset = gdf6[(gdf6['gt'] == 30) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 3)& (gdf6['cycle'] == 2)]
# subset = gdf6[(gdf6['gt'] == 60) & (gdf6['rgt'] == 218)]
# subset 


In [9]:
## length=10
subset2 =gdf6[(gdf6['gt'] == 10) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 1)& (gdf6['cycle'] == 4)]
# subset3 =gdf6[(gdf6['gt'] == 20) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 18)]
subset4 =gdf6[(gdf6['gt'] == 30) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 3)& (gdf6['cycle'] == 4)]
# subset5 =gdf6[(gdf6['gt'] == 40) & (gdf6['rgt'] == 218)& (gdf6['spot']== 4)& (gdf6['cycle'] == 4)]
subset6 =gdf6[(gdf6['gt'] == 50) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 4)]
# subset7 =gdf6[(gdf6['gt'] == 60) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 6)& (gdf6['cycle'] == 4)]



# ## length=20
# subset2 =gdf6[(gdf6['gt'] == 10) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 1)& (gdf6['cycle'] == 2)]
# subset3 =gdf6[(gdf6['gt'] == 20) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 18)]
# subset4 =gdf6[(gdf6['gt'] == 30) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 3)& (gdf6['cycle'] == 2)]
# # subset5 =gdf6[(gdf6['gt'] == 40) & (gdf6['rgt'] == 218)& (gdf6['spot']== 4)& (gdf6['cycle'] == 4)]
# subset6 =gdf6[(gdf6['gt'] == 50) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 4)]
# # subset7 =gdf6[(gdf6['gt'] == 60) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 6)& (gdf6['cycle'] == 4)]


# ## length=30
# subset2 =gdf6[(gdf6['gt'] == 10) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 1)& (gdf6['cycle'] == 2)]
# subset3 =gdf6[(gdf6['gt'] == 20) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 18)]
# subset4 =gdf6[(gdf6['gt'] == 30) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 3)& (gdf6['cycle'] == 2)]
# subset5 =gdf6[(gdf6['gt'] == 40) & (gdf6['rgt'] == 218)& (gdf6['spot']== 4)& (gdf6['cycle'] == 4)]
# subset6 =gdf6[(gdf6['gt'] == 50) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 4)]
# # subset7 =gdf6[(gdf6['gt'] == 60) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 6)& (gdf6['cycle'] == 4)]

# ## length=40
# subset2 =gdf6[(gdf6['gt'] == 10) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 1)& (gdf6['cycle'] == 2)]
# subset3 =gdf6[(gdf6['gt'] == 20) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 18)]
# subset4 =gdf6[(gdf6['gt'] == 30) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 3)& (gdf6['cycle'] == 2)]
# subset5 =gdf6[(gdf6['gt'] == 40) & (gdf6['rgt'] == 218)& (gdf6['spot']== 4)& (gdf6['cycle'] == 4)]
# subset6 =gdf6[(gdf6['gt'] == 50) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 5)& (gdf6['cycle'] == 4)]
# # subset7 =gdf6[(gdf6['gt'] == 60) & (gdf6['rgt'] == 218)& (gdf6['spot'] == 6)& (gdf6['cycle'] == 4)]



In [10]:
# gdf03 = icesat2.atl03sp({"poly": region, "rgt": 218, "cycle": 2, "len": length, "res": 20, "pass_invalid": True})
# gdf03x = sliderule.run("atl03x", {"poly": region, "rgt": 218, "cycle": 2})
gdf03x1 = sliderule.run("atl03x", {"poly": region, "rgt": 218, "srt": -1,"cnf": -1, "cycle": 2})    
gdf03x2 = sliderule.run("atl03x", {"poly": region, "rgt": 218, "srt": -1,"cnf": -1, "cycle": 4})
gdf03x3 = sliderule.run("atl03x", {"poly": region, "rgt": 218, "srt": -1,"cnf": -1, "cycle": 18})
# gdf03x


In [11]:
## length=10
data_pairs = [
    (subset2, gdf03x2),
    (subset4, gdf03x2),
    (subset6, gdf03x2),
]

# ## length=20
# data_pairs = [
#     (subset2, gdf03x1),
#     (subset3, gdf03x3),
#     (subset4, gdf03x1),
#     (subset6, gdf03x2),
# ]


# ## length=30
# data_pairs = [
#     (subset2, gdf03x1),
#     (subset3, gdf03x3),
#     (subset4, gdf03x1),
#     (subset5, gdf03x2),
#     (subset6, gdf03x2),
# ]


# ## length=40
# data_pairs = [
#     (subset2, gdf03x1),
#     (subset3, gdf03x3),
#     (subset4, gdf03x1),
#     (subset5, gdf03x2),
#     (subset6, gdf03x2),
# ]


In [16]:
def plot_elevation_photon(subset, gdf03x, cnf_colors,length):
    gt_value = subset['gt'].unique()[0]

    # Create plot
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot all ATL03 photons, grouped by confidence level
    for cnf_val in sorted(gdf03x['atl03_cnf'].dropna().unique()):
        color = cnf_colors.get(cnf_val, 'black')
        label = f'Photon cnf={cnf_val}'
        subset_cnf = gdf03x[gdf03x['atl03_cnf'] == cnf_val]
        ax.scatter(subset_cnf['x_atc'], subset_cnf['height'], s=0.5, color=color, label=label, alpha=0.6)

    # Plot ATL06 track
    ax.scatter(subset['x_atc'], subset['h_mean'], s=0.8, color='red',
               label=f"ATL06 spot={subset['spot'].unique()[0]}")

    # Customize plot
    ax.set_xlabel("Index along track (x_atc)")
    ax.set_ylabel("h_mean (m)")
    ax.set_title(
        f"ATL06 h_mean and ATL03 photons for length={length},GT={gt_value}, RGT={subset['rgt'].unique()[0]}, cycle={subset['cycle'].unique()[0]}"
    )
    ax.set_ylim([0, 80])
    ax.grid(True)
    ax.legend(markerscale=4)
    plt.tight_layout()
    plt.show()


In [17]:
cnf_colors = {
    0: 'gray',     # Noise
    1: 'blue',     # Buffer
    2: 'green',    # Low confidence
    3: 'orange',   # Medium confidence
    4: 'pink'       # High confidence
}



# Mapping each ATL06 subset with its corresponding ATL03 photon data
# data_pairs = [
#     (subset2, gdf03x2),
#     (subset3, gdf03x3),
#     (subset4, gdf03x2),
#     (subset5, gdf03x2),
#     (subset6, gdf03x2),
#     (subset7, gdf03x2),
# ]


# # Loop and pass each pair to the plotting function
# for atl06_subset, gdf03x_photons in data_pairs:
#     plot_elevation_photon(atl06_subset, gdf03x_photons, cnf_colors,length)




In [19]:
from pptx.util import Inches
import matplotlib.pyplot as plt
import os
from pptx import Presentation



def plot_elevation_photon1(subset, gdf03x, cnf_colors, length, prs):
    gt_value = subset['gt'].unique()[0]

    # Create and save the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    for cnf_val in sorted(gdf03x['atl03_cnf'].dropna().unique()):
        color = cnf_colors.get(cnf_val, 'black')
        ax.scatter(gdf03x[gdf03x['atl03_cnf'] == cnf_val]['x_atc'],
                   gdf03x[gdf03x['atl03_cnf'] == cnf_val]['height'],
                   s=0.5, color=color, alpha=0.6, label=f"Photon cnf={cnf_val}")
    ax.scatter(subset['x_atc'], subset['h_mean'], s=0.8, color='red',
               label=f"ATL06 spot={subset['spot'].unique()[0]}")
    ax.set_xlabel("x_atc")
    ax.set_ylabel("h_mean (m)")
    # ax.set_title(f"GT={gt_value}, RGT={subset['rgt'].unique()[0]}, Cycle={subset['cycle'].unique()[0]}")
    ax.set_title(f"ATL06 h_mean and ATL03 photons for length={length},GT={gt_value}, RGT={subset['rgt'].unique()[0]}, cycle={subset['cycle'].unique()[0]}")    
    ax.set_ylim([0, 80])
    ax.grid(True)
    ax.legend(markerscale=4)
    plt.tight_layout()

    plot_filename = f"gt_{gt_value}_plot.png"
    fig.savefig(plot_filename, dpi=300, bbox_inches='tight')
    plt.close(fig)

    # Add slide to existing presentation
    slide = prs.slides.add_slide(prs.slide_layouts[5])
    slide.shapes.add_picture(plot_filename, Inches(1), Inches(1), width=Inches(8))

    os.remove(plot_filename)



# # Create or open a PowerPoint presentation
# pptx_filename = "/home/jovyan/Glacier_preprocessing/Antarctica/latest_updatedcodes/ATL06_ATL03_Photon_Comparison.pptx"

# prs = Presentation(pptx_filename)

# for atl06_subset, gdf03x_photons in data_pairs:
#     plot_elevation_photon1(atl06_subset, gdf03x_photons, cnf_colors, length, prs)

# prs.save(pptx_filename)